#### pip install spacy openpyxl
#### python -m spacy download en_core_web_md


In [16]:
import pandas as pd

# Load your CSV files
categories_df = pd.read_excel("data/business_category_taxonomy.xlsx", engine='openpyxl')
domains_df = pd.read_excel("data/Naics3_taxonomy.xlsx", engine='openpyxl')

In [17]:
import spacy

# Load the medium spaCy model
nlp = spacy.load("en_core_web_md")

# Process descriptions to create vectors
categories_df["description_vector"] = categories_df["description"].apply(
    lambda desc: nlp(desc)
)
domains_df["description_vector"] = domains_df["description"].apply(
    lambda desc: nlp(desc)
)

OSError: [E050] Can't find model 'en_core_web_lg'. It doesn't seem to be a Python package or a valid path to a data directory.

In [ ]:
def find_best_match(cat_desc_vec, label_vectors):
    # Calculate similarities and return the index of the highest similarity
    similarities = [
        cat_desc_vec.similarity(label_desc) for label_desc in label_vectors
    ]
    return similarities.index(max(similarities))


# Convert domain vectors to a list for efficiency
label_vectors = domains_df["description_vector"].tolist()

# Find the best match for each category
categories_df["best_match_label_index"] = categories_df["description_vector"].apply(
    lambda x: find_best_match(x, label_vectors)
)

# Map the index to the actual domain name
categories_df["best_match_label"] = categories_df["best_match_label_index"].apply(
    lambda x: domains_df.iloc[x]["naics_label"]
)

In [ ]:
categories_df

,label,description,description_vector,best_match_label_index,best_match_label
0,ATVs Dealers & Services,Businesses categorized under ATVs Dealers & Se...,"(Businesses, categorized, under, ATVs, Dealers...",68,Rental and Leasing Services
1,Abortion Clinics,Abortion Clinics provide medical services rela...,"(Abortion, Clinics, provide, medical, services...",76,Hospitals
2,Accounting & Bookkeeping Services,Accounting & Bookkeeping Services encompass a ...,"(Accounting, &, Bookkeeping, Services, encompa...",74,Educational Services
3,Acupuncture clinic,An Acupuncture clinic is a healthcare facility...,"(An, Acupuncture, clinic, is, a, healthcare, f...",76,Hospitals
4,Adhesives & Sealants,Adhesives & Sealants encompass a diverse range...,"(Adhesives, &, Sealants, encompass, a, diverse...",26,Fabricated Metal Product Manufacturing
...,...,...,...,...,...
598,Work Clothing & Protection Equipment,Work Clothing & Protection Equipment pertains ...,"(Work, Clothing, &, Protection, Equipment, per...",55,Warehousing and Storage
599,Writers & Copywriters,Writers & Copywriters are skilled professional...,"(Writers, &, Copywriters, are, skilled, profes...",57,Publishing Industries
600,YMCA Camps,YMCA Camps offer structured programs and activ...,"(YMCA, Camps, offer, structured, programs, and...",86,"Religious, Grantmaking, Civic, Professional, a..."
601,Yoga Studios,Yoga Studios offer a tranquil space for indivi...,"(Yoga, Studios, offer, a, tranquil, space, for...",51,Scenic and Sightseeing Transportation
